# A Population Model

This section introduces a simplified **population model** designed to illustrate how *ModelFlow* handles **cohort dynamics**, **age transitions**, and **causal dependencies**.

The model tracks population changes across age groups and sexes, incorporating key demographic processes such as:

- **Births** — introducing new individuals into the youngest cohort  
- **Deaths** — removing individuals from each age group based on age-specific mortality rates  
- **Migration** — adjusting population levels through net inflows or outflows  
- **Aging** — transferring individuals from one age group to the next over time  

Through automatic list expansion and dependency tracing, *ModelFlow* enables clear and compact equation definitions while maintaining full transparency of the underlying causal structure.

In [1]:
#from pprint import pprint
from IPython.display import display, Math, Latex, Markdown , Image, SVG, display_svg,IFrame    
import pandas as pd

from modelconstruct import Mexplode,mfmod_list_to_codeblock


No modelwidgets


In [2]:
%load_ext autoreload
%autoreload 2

## List and sublist

Derived sublists
To support **modeling of cohorts** and **debt/asset developments**, *ModelFlow* automatically creates a set of derived sublists based on an initial list definition.

For example, the statement:

> $list\: \underbrace{ages}_{listname} = \underbrace{ages}_{sub listname} : age\_0 * age\_3$

defines a base list and generates several auxiliary lists that make it easier to model age transitions and boundary conditions.

The following sublists become available:

| **Key**          | **Description / Interpretation**              | **Values**                 |
|------------------|-----------------------------------------------|----------------------------|
| `AGES`           | The base list of age variables                | `['AGE_0', 'AGE_1', 'AGE_2']` |
| `AGES_AFTER`     | Elements shifted one step forward (next age)  | `['AGE_1', 'AGE_2', '0']`     |
| `AGES_BEFORE`    | Elements shifted one step backward (previous age) | `['0', 'AGE_0', 'AGE_1']` |
| `AGES_END`       | Flag: 1 if last element, else 0               | `['0', '0', '1']`          |
| `AGES_MIDDLE`    | Flag: 1 if middle element(s), else 0          | `['0', '1', '0']`          |
| `AGES_NOEND`     | Flag: 1 if not the last element, else 0       | `['1', '1', '0']`          |
| `AGES_NOSTART`   | Flag: 1 if not the first element, else 0      | `['0', '1', '1']`          |
| `AGES_START`     | Flag: 1 if first element, else 0              | `['1', '0', '0']`          |

### Example usage

These derived lists can be used both for **shifting variables** across age or maturity groups and for **conditional equation generation**.

**Recall** the definition of the doable expression

$$
\texttt{doable} \;
\underbrace{\langle tag \rangle}_{\text{Tag}} \;
\underbrace{[list\;\; sublist=value,\ldots]}_{\text{Optional Conditions}} \;
\underbrace{%
   \underbrace{\texttt{VARIABLE}}_{\text{LHS Expression}}
   \underbrace{\_\_\{list1\}\_\_\{list2\}...}_{\text{LHS Dimensions}}
}_{\text{LHS}} \;
= \;
\underbrace{\texttt{Expression including sublists}}_{\text{RHS Expression}}
$$



For instance, the statement below describes that the **population at the start of a year (`pop_primo`)** equals the **population in the same age group at the end of the previous year (`pop`)**, but **only for age groups that are not the first**:


>doable [ages ages_nostart]  pop_primo__{ages} = pop__{ages_before}

Here:

 - ages iterates over all defined age groups (AGE_0, AGE_1, AGE_2).

 - ages_nostart ensures that the equation is not applied to the first group (AGE_0).

 - ages_before provides the corresponding previous age ( AGE_1 → AGE_0, etc.).

This structure allows concise specification of interlinked cohort or maturity dynamics while automatically handling boundary conditions such as the first or last group.

In [3]:
Mexplode('''
>list ages = ages : age_0 * age_5
>doable [ages=ages_nostart]  pop_primo__{ages} = pop__{ages_before}(-1)''').show

FRML <> POP_PRIMO__AGE_1 = POP__AGE_0(-1) $
FRML <> POP_PRIMO__AGE_2 = POP__AGE_1(-1) $
FRML <> POP_PRIMO__AGE_3 = POP__AGE_2(-1) $
FRML <> POP_PRIMO__AGE_4 = POP__AGE_3(-1) $


## Specification of a model describing population dynamics

Below, a small population model is defined.  
To make the specification more concise and readable, two string replacements are performed through the call to `Mexplode`.

The replacements are:

| **Placeholder** | **Replaced with**                         | **Description** |
|:------------------|:-------------------------------------------|:-----------------|
| `__ab`           | `__{sexes}__{ages_before}`                | Refers to the same sex but the previous age group |
| `__a`            | `__{sexes}__{ages}`                       | Refers to the same sex and the current age group |

This approach reduces visual clutter in the equation definitions and helps express structured relationships more clearly across **sex** and **age** dimensions.


In [4]:
eq = r'''
# A Demography Model - Detailed Explanation

This model describes the evolution of a population over time, divided by age groups and sex.
It captures the fundamental demographic dynamics - births, deaths, and migration - which together determine 
how the total population changes from one period to the next.


---

## 1. List Definitions

>list ages = ages : age_0 * age_101  
>list sexes = sexes   : female male /  
>             fertile :     1     0  

These lists define the model’s structural dimensions:

- "ages" enumerates age groups, from age_0 (newborns) up to age_100 (the oldest group).  
- "sexes" distinguishes between female and male.  
- The "fertile" flag marks whether a sex is fertile (1 for females, 0 for males), which determines participation in the fertility equations.

---

## 2. Population Dynamics

The population is tracked at various stages within each period.

### a. Primo Population

>doable [ages=ages_nostart]  pop_primo__a = pop__ab(-1)

For all age groups except the youngest (`ages_nostart` excludes the start group),
the beginning-of-period (primo) population in an age group is equal to the end-of-period (ultimo) population of the previous age group in 
the previous period.
This represents aging: people move from one age group to the next as time progresses.

### b. Population Before Death

For the youngest age group:

>doable [ages=ages_start]   pop_before_death__a   = birth__total__{sexes} + migration__a

The youngest group’s population before death depends on births of that sex in the current period and net migration for that age group.
This reflects that new members of the population come from births and migration, since there is no prior age group for newborns.

For all other age groups:

>doable [ages=ages_nostart] pop_before_death__a   = pop_primo__a + migration__a

Older groups’ populations before death are determined by the primo population (those who aged into the group) and migration inflows or outflows.

---

## 3. Mortality and Deaths

>doable  [ages=ages_end ]  mortality__a =1.0  
>doable  <sum=total> death__a = pop_before_death__a  * mortality__a

For the last age group, the death rate is set to 1.0, implying that everyone in this group dies within the period - 
a closure condition preventing population overflow beyond the oldest cohort.
Deaths in all groups are computed as the product of the population before death and the age-specific death rate.

## 4. End-of-Period (Ultimo) Population

>doable  <sum=total> pop__a  = pop_before_death__a  - death__a

The end-of-period population equals the population before death minus deaths.
This is the surviving population that will age into the next group in the following period.


## 5. Births and Fertility

>doable <sum=total> [ages=ages_nostart,  sexes=fertile] birth__a = pop_primo__a * fertility__a

Births are generated by fertile females in each age group.
Fertility rates (fertility__a) vary by age, and are in the dataframe set to zero outside reproductive ages.
The total number of births is thus the sum across fertile age groups.

---

## 6. Sex Ratio at Birth

>birth__total__female = FRAC_BIRTH__female * birth__total  
>birth__total__male = (1 - FRAC_BIRTH__female) * birth__total

The total births are split into female and male newborns using the fraction `FRAC_BIRTH__female`.
This ensures a biologically realistic sex distribution at birth.

>total_fertility_rate = sum(ages,sum(sexes,fertility__a))

---

## 7. Totals and Consistency Check

>migration__total = sum(ages,sum(sexes,migration__a))  
migration__total aggregates migration flows over all ages and sexes.

>pop__total_check  = (pop__total(-1) + birth__total + migration__total - death__total) - pop__total

pop__total_check serves as a consistency test for the model:
The change in total population should equal births + migration – deaths.
Any deviation indicates potential modeling or data inconsistencies.

---

## Summary

This demographic model captures the fundamental accounting identities of population dynamics:

Population<sub>t</sub><sup>age</sup> = Population<sub>t-1</sub><sup>age-1</sup> + Births<sub>t</sub><sup>age</sup> + Migration<sub>t</sub><sup>age</sup> - Deaths<sub>t</sub><sup>age</sup>

For the youngest cohort (age = 0), there is no inflow from age − 1,  
so the equation simplifies to:

Population<sub>t</sub><sup>0</sup> = Births<sub>t</sub> + Migration<sub>t</sub><sup>0</sup> - Deaths<sub>t</sub><sup>0</sup>

It distinguishes individuals by age and sex, allowing for realistic aging, fertility, and mortality processes.
The model is modular and extendable - one could add features such as:
- Age-dependent migration patterns
- Variable mortality shocks (for example, pandemics)
- Policy impacts on fertility rates
- Regional or socioeconomic subpopulations

'''
mm = Mexplode(eq,
replacements=[(r'__ab',r'__{sexes}__{ages_before}'),(r'__a',r'__{sexes}__{ages}')])

## From This Input:

In [5]:
mm.render


# A Demography Model - Detailed Explanation

This model describes the evolution of a population over time, divided by age groups and sex.
It captures the fundamental demographic dynamics - births, deaths, and migration - which together determine 
how the total population changes from one period to the next.


---

## 1. List Definitions

```text
>list ages = ages : age_0 * age_101  
>list sexes = sexes   : female male /  
>             fertile :     1     0  
```

These lists define the model’s structural dimensions:

- "ages" enumerates age groups, from age_0 (newborns) up to age_100 (the oldest group).  
- "sexes" distinguishes between female and male.  
- The "fertile" flag marks whether a sex is fertile (1 for females, 0 for males), which determines participation in the fertility equations.

---

## 2. Population Dynamics

The population is tracked at various stages within each period.

### a. Primo Population

```text
>doable [ages=ages_nostart]  pop_primo__a = pop__ab(-1)
```

For all age groups except the youngest (`ages_nostart` excludes the start group),
the beginning-of-period (primo) population in an age group is equal to the end-of-period (ultimo) population of the previous age group in 
the previous period.
This represents aging: people move from one age group to the next as time progresses.

### b. Population Before Death

For the youngest age group:

```text
>doable [ages=ages_start]   pop_before_death__a   = birth__total__{sexes} + migration__a
```

The youngest group’s population before death depends on births of that sex in the current period and net migration for that age group.
This reflects that new members of the population come from births and migration, since there is no prior age group for newborns.

For all other age groups:

```text
>doable [ages=ages_nostart] pop_before_death__a   = pop_primo__a + migration__a
```

Older groups’ populations before death are determined by the primo population (those who aged into the group) and migration inflows or outflows.

---

## 3. Mortality and Deaths

```text
>doable  [ages=ages_end ]  mortality__a =1.0  
>doable  <sum=total> death__a = pop_before_death__a  * mortality__a
```

For the last age group, the death rate is set to 1.0, implying that everyone in this group dies within the period - 
a closure condition preventing population overflow beyond the oldest cohort.
Deaths in all groups are computed as the product of the population before death and the age-specific death rate.

## 4. End-of-Period (Ultimo) Population

```text
>doable  <sum=total> pop__a  = pop_before_death__a  - death__a
```

The end-of-period population equals the population before death minus deaths.
This is the surviving population that will age into the next group in the following period.


## 5. Births and Fertility

```text
>doable <sum=total> [ages=ages_nostart,  sexes=fertile] birth__a = pop_primo__a * fertility__a
```

Births are generated by fertile females in each age group.
Fertility rates (fertility__a) vary by age, and are in the dataframe set to zero outside reproductive ages.
The total number of births is thus the sum across fertile age groups.

---

## 6. Sex Ratio at Birth

```text
>birth__total__female = FRAC_BIRTH__female * birth__total  
>birth__total__male = (1 - FRAC_BIRTH__female) * birth__total
```

The total births are split into female and male newborns using the fraction `FRAC_BIRTH__female`.
This ensures a biologically realistic sex distribution at birth.

```text
>total_fertility_rate = sum(ages,sum(sexes,fertility__a))
```

---

## 7. Totals and Consistency Check

```text
>migration__total = sum(ages,sum(sexes,migration__a))  
```
migration__total aggregates migration flows over all ages and sexes.

```text
>pop__total_check  = (pop__total(-1) + birth__total + migration__total - death__total) - pop__total
```

pop__total_check serves as a consistency test for the model:
The change in total population should equal births + migration – deaths.
Any deviation indicates potential modeling or data inconsistencies.

---

## Summary

This demographic model captures the fundamental accounting identities of population dynamics:

Population<sub>t</sub><sup>age</sup> = Population<sub>t-1</sub><sup>age-1</sup> + Births<sub>t</sub><sup>age</sup> + Migration<sub>t</sub><sup>age</sup> - Deaths<sub>t</sub><sup>age</sup>

For the youngest cohort (age = 0), there is no inflow from age − 1,  
so the equation simplifies to:

Population<sub>t</sub><sup>0</sup> = Births<sub>t</sub> + Migration<sub>t</sub><sup>0</sup> - Deaths<sub>t</sub><sup>0</sup>

It distinguishes individuals by age and sex, allowing for realistic aging, fertility, and mortality processes.
The model is modular and extendable - one could add features such as:
- Age-dependent migration patterns
- Variable mortality shocks (for example, pandemics)
- Policy impacts on fertility rates
- Regional or socioeconomic subpopulations


## This high level model is extracted:

In [6]:
print(mm.clean_frml_statements)

LIST AGES =
    AGES : AGE_0 * AGE_101  $
LIST SEXES =
    SEXES   : FEMALE MALE /
    FERTILE : 1      0  $
DOABLE [AGES=AGES_NOSTART]  POP_PRIMO__{SEXES}__{AGES} = POP__{SEXES}__{AGES_BEFORE}(-1)$
DOABLE [AGES=AGES_START]   POP_BEFORE_DEATH__{SEXES}__{AGES}   = BIRTH__TOTAL__{SEXES} + MIGRATION__{SEXES}__{AGES}$
DOABLE [AGES=AGES_NOSTART] POP_BEFORE_DEATH__{SEXES}__{AGES}   = POP_PRIMO__{SEXES}__{AGES} + MIGRATION__{SEXES}__{AGES}$
DOABLE  [AGES=AGES_END ]  MORTALITY__{SEXES}__{AGES} =1.0  $
DOABLE  <SUM=TOTAL> DEATH__{SEXES}__{AGES} = POP_BEFORE_DEATH__{SEXES}__{AGES}  * MORTALITY__{SEXES}__{AGES}$
DOABLE  <SUM=TOTAL> POP__{SEXES}__{AGES}  = POP_BEFORE_DEATH__{SEXES}__{AGES}  - DEATH__{SEXES}__{AGES}$
DOABLE <SUM=TOTAL> [AGES=AGES_NOSTART,  SEXES=FERTILE] BIRTH__{SEXES}__{AGES} = POP_PRIMO__{SEXES}__{AGES} * FERTILITY__{SEXES}__{AGES}$
FRML  <> BIRTH__TOTAL__FEMALE = FRAC_BIRTH__FEMALE * BIRTH__TOTAL   $
FRML  <> BIRTH__TOTAL__MALE = (1 - FRAC_BIRTH__FEMALE) * BIRTH__TOTAL $
FRML  <

## Then the doable statements are expanded:

In [7]:
print(mm.post_doable)

LIST AGES =
    AGES : AGE_0 * AGE_101  $
LIST SEXES =
    SEXES   : FEMALE MALE /
    FERTILE : 1      0  $
DO SEXES  $ DO AGES AGES_NOSTART = 1 $
  FRML    <> POP_PRIMO__{SEXES}__{AGES} = POP__{SEXES}__{AGES_BEFORE}(-1)$ $
ENDDO $ ENDDO $  
DO SEXES  $ DO AGES AGES_START = 1 $
  FRML    <> POP_BEFORE_DEATH__{SEXES}__{AGES}   = BIRTH__TOTAL__{SEXES} + MIGRATION__{SEXES}__{AGES}$ $
ENDDO $ ENDDO $  
DO SEXES  $ DO AGES AGES_NOSTART = 1 $
  FRML    <> POP_BEFORE_DEATH__{SEXES}__{AGES}   = POP_PRIMO__{SEXES}__{AGES} + MIGRATION__{SEXES}__{AGES}$ $
ENDDO $ ENDDO $  
DO SEXES  $ DO AGES AGES_END  = 1 $
  FRML    <> MORTALITY__{SEXES}__{AGES} =1.0  $ $
ENDDO $ ENDDO $  
DO SEXES  $ DO AGES  $
  FRML    <SUM=TOTAL> DEATH__{SEXES}__{AGES} = POP_BEFORE_DEATH__{SEXES}__{AGES}  * MORTALITY__{SEXES}__{AGES}$ $
ENDDO $ ENDDO $ 
FRML <SUM=TOTAL> DEATH__TOTAL = SUM(SEXES,SUM(AGES,DEATH__{SEXES}__{AGES})) $ 

 
DO SEXES  $ DO AGES  $
  FRML    <SUM=TOTAL> POP__{SEXES}__{AGES}  = POP_BEFORE_DEATH__{SE

## This low level model is generated:

In [8]:
mm.show

FRML <> POP_PRIMO__FEMALE__AGE_1 = POP__FEMALE__AGE_0(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_2 = POP__FEMALE__AGE_1(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_3 = POP__FEMALE__AGE_2(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_4 = POP__FEMALE__AGE_3(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_5 = POP__FEMALE__AGE_4(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_6 = POP__FEMALE__AGE_5(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_7 = POP__FEMALE__AGE_6(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_8 = POP__FEMALE__AGE_7(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_9 = POP__FEMALE__AGE_8(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_10 = POP__FEMALE__AGE_9(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_11 = POP__FEMALE__AGE_10(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_12 = POP__FEMALE__AGE_11(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_13 = POP__FEMALE__AGE_12(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_14 = POP__FEMALE__AGE_13(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_15 = POP__FEMALE__AGE_14(-1) $
FRML <> POP_PRIMO__FEMALE__AGE_16 = POP__FEMALE__AGE_15(-1) $
FRML <> POP_PRIMO__FEMALE__

## Tracing the causality structure

In the example below, the **causal chain** leading to the variable `POP__MALE__AGE_2` is traced upward through **8 dependency levels**.

>mm.mmodel.pop__male__age_2.tracepre(up=8, size=(10,10))   

This produces a causal dependency graph (as shown below), illustrating how upstream variables contribute to the population of 2-year-old males:

 - Blue boxes represent endogenous population and mortality variables.

 - Yellow boxes indicate exogenous , such as migration and death rates.

 - The red box highlights the target variable (POP__MALE__AGE_2).

The arrows show the flow of causality, moving from fundamental drivers like births, migration, and mortality rates toward the resulting population cohort.
This visualization helps understand how demographic processes propagate through the model and affect specific age groups.

In [9]:
mm.mmodel.pop__male__age_3.tracepre(up=8,size=(10,10))

In [10]:
exo_df = pd.read_parquet("data/exo_population.parquet")


In [11]:
mpopulation = mm.mmodel

In [12]:
mpopulation

<
Model name                              :            testmodel 
Model structure                         :            Recursive 
Number of variables                     :                 1621 
Number of exogeneous  variables         :                  605 
Number of endogeneous variables         :                 1016 
>

In [13]:
res = mpopulation(exo_df,silent=True)

In [14]:
mpopulation['POP__total']

In [15]:
mpopulation.POP__FEMALE__AGE_0.tracepre(up=3)  

In [16]:
mpopulation['birth_*total*']

In [17]:
mpopulation.POP__TOTAL_CHECK  

Endogeneous: POP__TOTAL_CHECK: 
Formular: FRML <> POP__TOTAL_CHECK = (POP__TOTAL(-1)+BIRTH__TOTAL+MIGRATION__TOTAL-DEATH__TOTAL)-POP__TOTAL $

POP__TOTAL_CHECK: 
BIRTH__TOTAL    : 
DEATH__TOTAL    : 
MIGRATION__TOTAL: 
POP__TOTAL      : 

Values :


YEAR,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124
Base,"-6,263,325.00",0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00
Last,"-6,263,325.00",0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,0.00,0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,0.00,0.00,-0.00,0.00,0.00,-0.00,-0.00,0.00,-0.00,0.00,0.00,-0.00,0.00,-0.00,-0.00,-0.00,0.00
Diff,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


Input last run:


YEAR,2026,2027,2028,2029,2030,2031,2032,2033,2034,2035,2036,2037,2038,2039,2040,2041,2042,2043,2044,2045,2046,2047,2048,2049,2050,2051,2052,2053,2054,2055,2056,2057,2058,2059,2060,2061,2062,2063,2064,2065,2066,2067,2068,2069,2070,2071,2072,2073,2074,2075,2076,2077,2078,2079,2080,2081,2082,2083,2084,2085,2086,2087,2088,2089,2090,2091,2092,2093,2094,2095,2096,2097,2098,2099,2100,2101,2102,2103,2104,2105,2106,2107,2108,2109,2110,2111,2112,2113,2114,2115,2116,2117,2118,2119,2120,2121,2122,2123,2124
BIRTH__TOTAL,"59,130.94","58,694.01","58,194.78","57,630.94","57,003.11","56,309.70","55,558.69","54,764.12","53,946.49","53,134.81","52,355.52","51,634.53","50,989.15","50,426.96","49,953.40","49,566.17","49,263.75","49,037.11","48,876.79","48,771.99","48,711.19","48,683.06","48,673.74","48,672.26","48,665.86","48,645.27","48,604.50","48,535.02","48,432.51","48,293.56","48,109.77","47,880.31","47,599.27","47,269.00","46,891.32","46,470.45","46,011.61","45,522.78","45,013.85","44,494.09","43,975.18","43,466.62","42,977.14","42,516.42","42,090.80","41,705.13","41,362.63","41,064.01","40,808.50","40,593.74","40,415.76","40,269.61","40,149.43","40,048.79","39,961.02","39,879.45","39,797.63","39,709.49","39,609.49","39,492.73","39,355.12","39,193.38","39,005.30","38,789.66","38,546.39","38,276.49","37,982.02","37,666.05","37,332.43","36,985.72","36,630.91","36,273.26","35,918.01","35,570.20","35,234.46","34,914.80","34,614.44","34,335.80","34,080.33","33,848.58","33,640.16","33,453.86","33,287.71","33,139.10","33,004.92","32,881.71","32,765.80","32,653.43","32,540.93","32,424.83","32,301.95","32,169.57","32,025.43","31,867.86","31,695.80","31,508.80","31,307.08","31,091.42","30,863.19"
DEATH__TOTAL,"107,778.51","104,295.37","101,135.67","98,207.64","95,874.72","93,922.32","92,074.89","90,680.95","89,591.29","88,621.08","87,795.39","87,023.68","86,405.86","85,761.63","85,190.16","84,599.74","83,957.98","83,284.11","82,632.40","81,964.65","81,286.49","80,593.77","79,972.08","79,312.83","78,759.93","78,184.20","77,630.81","77,171.79","76,716.97","76,331.23","75,976.53","75,595.76","75,275.64","75,024.59","74,787.30","74,581.17","74,414.93","74,268.33","74,187.47","74,059.58","74,011.15","73,898.13","73,853.54","73,805.53","73,784.62","73,757.28","73,730.69","73,629.16","73,508.30","73,372.79","73,212.96","73,049.82","72,734.51","72,467.48","72,155.98","71,849.89","71,446.41","70,990.66","70,490.05","69,951.43","69,381.60","68,783.69","68,165.98","67,534.40","66,893.19","66,251.03","65,611.19","64,981.82","64,364.34","63,767.46","63,197.49","62,659.16","62,155.12","61,691.06","61,263.29","60,866.51","60,494.10","60,147.22","59,822.16","59,509.65","59,206.90","58,911.30","58,611.29","58,294.49","57,950.83","57,594.43","57,210.68","56,793.15","56,359.06","55,897.24","55,421.51","54,930.89","54,432.49","53,928.46","53,425.07","52,925.16","52,433.32","51,952.83","51,484.81"
MIGRATION__TOTAL,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
POP__TOTAL,"6,214,677.44","6,169,076.08","6,126,135.19","6,085,558.49","6,046,686.88","6,009,074.26","5,972,558.07","5,936,641.23","5,900,996.43","5,865,510.15","5,830,070.28","5,794,681.14","5,759,264.43","5,723,929.76","5,688,693.00","5,653,659.44","5,618,965.20","5,584,718.21","5,550,962.60","5,517,769.94","5,485,194.64","5,453,283.93","5,421,985.60","5,391,345.02","5,361,250.95","5,331,712.02","5,302,685.70","5,274,048.93","5,245,764.47","5,217,726.80","5,189,860.03","5,162,144.58","5,134,468.21","5,106,712.62","5,078,816.63","5,050,705.91","5,022,302.60","4,993,557.06","4

In [18]:
mpopulation['TOTAL_FERTILITY_RATE']

In [19]:
mpopulation.modeldash('POP__FEMALE__AGE_0')

No Dash
No Dash, name 'DashInteractiveGraphviz' is not defined


In [20]:
mpopulation.modeldump('models/population')